In [ ]:
import pandas as pd
import numpy as np
import glob

: 

In [ ]:
df = pd.read_csv('estat_prc_hpi_q.csv')
df.columns = df.columns.str.strip()
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df.head()

: 

In [ ]:
path = '../ESTAT_CODELISTS/*.tsv'
files = glob.glob(path)
dataframes = {file.split('/')[-1].replace('.tsv', ''): pd.read_csv(
    file, sep='\t', on_bad_lines='skip') for file in files}

: 

In [ ]:
for col in ["freq", "unit", "purchase", "geo"]:
    curr_codelist = dataframes[f"ESTAT_{col.upper()}"]
    df[col] = df[col].map(curr_codelist.set_index('CODE')['Label'])

df.rename(columns={'geo': 'country'}, inplace=True)
df.head()

: 

In [ ]:
df.replace(": z", np.nan, inplace=True)
df.replace(":", np.nan, inplace=True)

: 

In [ ]:
year_cols = [col for col in df.columns if col.isdigit()]

for col in year_cols:
    df[col] = df[col].astype(str).str.extract(r'(\d+)').astype(float)

: 

In [ ]:
df.head()

: 

In [ ]:
for col in df.columns:
    if not col.isdigit() and df[col].nunique() == 1:
        df.drop(columns=[col], inplace=True)

: 

In [ ]:
df = df.applymap(
    lambda x: pd.NA if pd.isna(x) else 
    pd.Series(str(x)).str.replace(r'(\d+\.\d+|\d+)\s*[a-zA-Z]+\.?', r'\1', regex=True).iloc[0]
)

: 

In [ ]:
df.head()

: 

In [ ]:
df.to_csv("./clean/estat_prc_hpi_q.csv", index=False)

: 